In [ ]:
!pip -q install datasets transformers sentence-transformers evaluate shortuuid gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

import json

In [ ]:
# # Reformat data

# train_data_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/train.json'
# val_data_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/val.json'
# test_data_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/test.json'

# # load files
# def read_json(path):
#     with open(path) as f:
#         data = json.load(f)
#     return data

# train_json = read_json(train_data_path)
# val_json = read_json(val_data_path)
# test_json = read_json(test_data_path)

# def extract_data(json_data):
#     data = []
#     for i in range(len(json_data['data'])):
#         title = json_data['data'][i]['title']
#         for j in range(len(json_data['data'][i]['paragraphs'])):
#             context = json_data['data'][i]['paragraphs'][j]['context']
#             for k in range(len(json_data['data'][i]['paragraphs'][j]['qas'])):
#                 question = json_data['data'][i]['paragraphs'][j]['qas'][k]['question']
#                 answers = json_data['data'][i]['paragraphs'][j]['qas'][k]['answers']
#                 idx = json_data['data'][i]['paragraphs'][j]['qas'][k]['id']
#                 data.append({
#                     'version': json_data['version'],
#                     'title': title,
#                     'context': context,
#                     'question': question,
#                     'answers': {
#                         'text': [answer['text'] for answer in answers],
#                         'answer_start': [answer['answer_start'] for answer in answers]
#                     },
#                     'id': idx
#                 })
#     dataset = Dataset.from_dict({
#         'id': [d['id'] for d in data],
#         'title': [d['title'] for d in data],
#         'context': [d['context'] for d in data],
#         'question': [d['question'] for d in data],
#         'answers': [d['answers'] for d in data],
#         'version': [d['version'] for d in data]
#     })
#     return dataset

# train_ds = extract_data(train_json)
# val_ds = extract_data(val_json)
# test_ds = extract_data(test_json)

# train_ds.save_to_disk('/content/drive/MyDrive/Hust/viettel_project/data/viquad/train_ds')
# val_ds.save_to_disk('/content/drive/MyDrive/Hust/viettel_project/data/viquad/val_ds')
# test_ds.save_to_disk('/content/drive/MyDrive/Hust/viettel_project/data/viquad/test_ds')

In [ ]:
train_ds_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/train_ds'
val_ds_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/val_ds'
test_ds_path = '/content/drive/MyDrive/Hust/viettel_project/data/viquad/test_ds'

train_origin = Dataset.load_from_disk(train_ds_path)
val_origin = Dataset.load_from_disk(val_ds_path)
test_origin = Dataset.load_from_disk(test_ds_path)

In [ ]:
test_origin.shuffle()[0]

{'id': 'uit_01__00061_7_2',
 'title': 'Đức',
 'context': 'Những quân chủ của Vương triều Otto (919–1024) hợp nhất một số công quốc lớn và Quốc vương người German/Đức Otto I đăng quang Hoàng đế La Mã Thần thánh của các khu vực này vào năm 962. Năm 996, Giáo hoàng Grêgôriô V trở thành giáo hoàng người Đức đầu tiên, do người họ hàng của ông là Otto III bổ nhiệm- không lâu sau đăng quang Hoàng đế La Mã Thần thánh. Đế quốc La Mã Thần thánh sáp nhập miền bắc Ý ngày nay và khu vực Bourgogne nay thuộc Pháp dưới thời trị vì của các hoàng đế thuộc Gia tộc Salier (1024–1125), song các hoàng đế mất đi quyền lực do tranh luận phong chức với giáo hội.',
 'question': 'Vị trí Giáo hoàng mà người Đức lần đầu tiên đảm nhiệm là vào năm nào?',
 'answers': {'answer_start': [185, 189, 194, 185],
  'text': ['Năm 996', '996', 'Giáo hoàng Grêgôriô V', 'Năm 996']},
 'version': '1.1'}

In [ ]:
train_origin[0]

{'id': 'uit_01__05272_0_1',
 'title': 'Phạm Văn Đồng',
 'context': 'Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).',
 'question': 'Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?',
 'answers': {'answer_start': [507], 'text': ['Lâm Bá Kiệt']},
 'version': '1.1'}

In [ ]:
reader = AutoModelForQuestionAnswering.from_pretrained('deepset/xlm-roberta-base-squad2')
tokenizer = AutoTokenizer.from_pretrained('deepset/xlm-roberta-base-squad2')

### Process input for model

In [ ]:
max_length = 256
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    # #test
    # idx = 0
    # sample_idx = sample_map[idx]
    # answer = answers[sample_idx]["text"][0] if len(answers[sample_idx]["text"]) > 0 else ""

    # start = start_positions[idx]
    # end = end_positions[idx]
    # labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1], skip_special_tokens=True)
    # print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
train_dataset = train_origin.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_origin.column_names,
)
val_dataset = val_origin.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_origin.column_names,
)
test_dataset = test_origin.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_origin.column_names,
)
train_dataset

Map:   0%|          | 0/18579 [00:00<?, ? examples/s]

Map:   0%|          | 0/2285 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 26378
})

In [ ]:
val_dataset

Map:   0%|          | 0/2285 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 3008
})

In [ ]:
test_dataset

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 3142
})

### Model Training

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from collections import defaultdict
import evaluate
from tqdm.auto import tqdm
metric = evaluate.load("squad")


from tqdm.auto import tqdm


# def compute_metrics(start_logits, end_logits, features, examples, n_best=5, max_answer_length=100):
#     example_to_features = defaultdict(list)
#     for idx, feature in enumerate(features):
#         example_to_features[feature["example_id"]].append(idx)

#     predicted_answers = []
#     for example in tqdm(examples):
#         example_id = example["id"]
#         context = example["context"]
#         answers = []

#         # Loop through all features associated with that example
#         for feature_index in example_to_features[example_id]:
#             start_logit = start_logits[feature_index]
#             end_logit = end_logits[feature_index]
#             offsets = features[feature_index]["offset_mapping"]

#             start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
#             end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
#             for start_index in start_indexes:
#                 for end_index in end_indexes:
#                     # Skip answers that are not fully in the context
#                     if offsets[start_index] is None or offsets[end_index] is None:
#                         continue
#                     # Skip answers with a length that is either < 0 or > max_answer_length
#                     if (
#                         end_index < start_index
#                         or end_index - start_index + 1 > max_answer_length
#                     ):
#                         continue

#                     answer = {
#                         "text": context[offsets[start_index][0] : offsets[end_index][1]],
#                         "logit_score": start_logit[start_index] + end_logit[end_index],
#                     }
#                     answers.append(answer)

#         # Select the answer with the best score
#         if len(answers) > 0:
#             best_answer = max(answers, key=lambda x: x["logit_score"])
#             predicted_answers.append(
#                 {"id": example_id, "prediction_text": best_answer["text"]}
#             )
#         else:
#             predicted_answers.append({"id": example_id, "prediction_text": ""})

#     theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
#     result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
#     print(result)
#     return result

def compute_metrics(
    start_logits,
    end_logits,
    features,
    examples,
    metric=metric,
    n_best=10,
    max_answer_length=200,
):
    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            candidate_answers = []
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if start_index == 0 and end_index == 0:
                        candidate_answers.append(
                            {
                                "text": "",
                                "logit_score": start_logit[start_index]
                                + end_logit[end_index],
                            }
                        )
                        continue

                    # Skip candidate_answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip candidate_answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    answer = {
                        "text": context[
                            offsets[start_index][0] : offsets[end_index][1]
                        ],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    candidate_answers.append(answer)

            # Add the best candidate_answers with a score > zero to the final list
            candidate_answers = sorted(
                candidate_answers, key=lambda x: x["logit_score"], reverse=True
            )
            answers.append(candidate_answers[0])

        # if all candidate_answers is "", then we will predict ""
        empty_count = sum([1 if ans["text"] == "" else 0 for ans in answers])
        if empty_count == len(answers):
            predicted_answers.append({"id": example_id, "prediction_text": ""})
        # else we will predict the best non-empty answer
        else:
            not_empty_answers = [ans for ans in answers if ans["text"] != ""]
            best_answer = max(not_empty_answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )

    theoretical_answers = [
        {
            "id": ex["id"],
            "answers": {
                "text": ex["answers"]["text"] if len(ex["answers"]["text"]) > 0 else [""],
                "answer_start": ex["answers"]["answer_start"] if len(ex["answers"]["text"]) > 0 else [0],
            },
        }
        for ex in examples
    ]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
check_point = "viettel-xlm-roberta-base-squad-finetune-viquad"
args = TrainingArguments(
    check_point,
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    push_to_hub=True,
)
trainer = Trainer(
    model=reader,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.426200
1000,1.260600
1500,1.224300
2000,1.144300
2500,1.137200
3000,1.079300


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


TrainOutput(global_step=3298, training_loss=1.201669566916147, metrics={'train_runtime': 1534.0365, 'train_samples_per_second': 17.195, 'train_steps_per_second': 2.15, 'total_flos': 3446242924591104.0, 'train_loss': 1.201669566916147, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

In [ ]:
predicitions, _, _ = trainer.predict(val_dataset)
start_logits, end_logits = predicitions
compute_metrics(start_logits, end_logits, val_dataset, val_origin)

  0%|          | 0/2285 [00:00<?, ?it/s]

{'exact_match': 65.86433260393873, 'f1': 82.47505929605575}

### Inference

In [ ]:
check_point = 'hoang14/viettel-xlm-roberta-base-squad-finetune-viquad'
reader = AutoModelForQuestionAnswering.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)

In [ ]:
predicitions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predicitions

In [ ]:
compute_metrics(start_logits, end_logits, test_dataset, test_origin)

  0%|          | 0/2210 [00:00<?, ?it/s]

{'exact_match': 66.87782805429865, 'f1': 83.62059021948447}

### Demo

In [ ]:
!pip -q install datasets transformers sentence-transformers evaluate shortuuid gradio

In [ ]:
def preprocess_examples(examples, tokenizer, max_length=420, stride=140):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

def get_predictions(start_logits, end_logits, features, examples, n_best=10, max_answer_length=100):
    from collections import defaultdict
    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            answers.sort(key=lambda x: x["logit_score"], reverse=True)
            answers = answers[:n_best]
            # normalize scores by taking softmax
            scores = np.array([ans["logit_score"] for ans in answers])
            scores = np.exp(scores) / np.exp(scores).sum()
            for ans, score in zip(answers, scores):
                ans['confidence'] = score
            predicted_answers.append({
                'id': example_id,
                'context': context,
                'question': example['question'],
                'answers': answers,
            })            
        else:
            predicted_answers.append(
                {
                    "id": example_id,
                    "context": context,
                    "question": example["question"],
                    "answers": [{"text": "", "logit_score": None, "confidence": None}],
                }
            )
    return predicted_answers

def predict(examples, tokenizer, model, max_length=420, stride=140):
    from shortuuid import uuid
    examples['id'] = [uuid() for _ in range(len(examples['context']))]
    features = preprocess_examples(examples, tokenizer, max_length=max_length, stride=stride)
    inputs = {
        "input_ids": torch.tensor(features["input_ids"]).to(model.device),
        "attention_mask": torch.tensor(features["attention_mask"]).to(model.device),
    }
    outputs = model(**inputs)
    start_logits = outputs.start_logits.detach().numpy()
    end_logits = outputs.end_logits.detach().numpy()
    features = [{k: v[i] for k, v in features.items()} for i in range(len(features["input_ids"]))]
    examples = [{k: v[i] for k, v in examples.items()} for i in range(len(examples["context"]))]
    return get_predictions(start_logits, end_logits, features, examples)

In [ ]:
check_point = 'hoang14/viettel-xlm-roberta-base-squad-finetune-viquad'
reader = AutoModelForQuestionAnswering.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)

In [ ]:
import gradio as gr

ip1 = gr.inputs.Textbox(lines=6, placeholder="Context", label="Context")
ip2 = gr.inputs.Textbox(lines=2, placeholder="Question", label="Question")
op = gr.outputs.Textbox(label="Answer")

def qa(context, question, model=reader, tokenizer=tokenizer):
  reader_inputs = {
    'question':[question],
    'context':[context]
  }
  answers = predict(reader_inputs, tokenizer, model)
  return answers[0]['answers'][0]['text']
  
iface = gr.Interface(
  fn=qa,
  inputs=[ip1, ip2],
  outputs=op,
  title="Vietnamese Question Answering",
  description="Demo for Vietnamese Question Answering"
)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6037d8b3ef074058a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
